### Fine-Tune BERT Model
Fine-tune the pre-trained BERT model on the news dataset.

In [ ]:
import tensorflow as tf
from transformers import TFBertModel
from sklearn.model_selection import train_test_split

def build_model():
    """
    Builds the multi-task BERT model for sentiment analysis and ranking.
    """
    # Load pre-trained BERT model
    bert_model = TFBertModel.from_pretrained('bert-base-uncased')

    # Define inputs
    input_ids = tf.keras.Input(shape=(128,), dtype='int32')
    attention_mask = tf.keras.Input(shape=(128,), dtype='int32')

    # BERT output
    bert_output = bert_model(input_ids, attention_mask=attention_mask)[1]  # Pooled output

    # Shared layers
    dense_1 = tf.keras.layers.Dense(64, activation="relu")(bert_output)
    dropout = tf.keras.layers.Dropout(0.2)(dense_1)

    # Task 1: Sentiment Analysis
    sentiment_output = tf.keras.layers.Dense(1, activation="sigmoid", name="sentiment_output")(dropout)

    # Task 2: Ranking Prediction
    rank_output = tf.keras.layers.Dense(1, activation="linear", name="rank_output")(dropout)

    # Define the model
    model = tf.keras.Model(inputs=[input_ids, attention_mask], outputs=[sentiment_output, rank_output])

    # Compile the model
    model.compile(
        optimizer=tf.keras.optimizers.AdamW(learning_rate=2e-5),
        loss={"sentiment_output": "binary_crossentropy", "rank_output": "mse"},
        metrics={"sentiment_output": "accuracy", "rank_output": "mae"}
    )

    return model